In [1]:
import pyspark
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("kaggle_hm_app").getOrCreate()

transactions_train_df = spark.read.option("header", True).csv("../00_data/transactions_train.csv")
articles_df = spark.read.option("header", True).csv("../00_data/articles.csv")
customers_df = spark.read.option("header", True).csv("../00_data/customers.csv")


22/03/03 11:30:11 WARN Utils: Your hostname, VM-DEV.sglk.local resolves to a loopback address: 127.0.0.1; using 10.0.2.15 instead (on interface enp0s3)
22/03/03 11:30:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/03 11:30:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [151]:
transaction_train_2018 = transactions_train_df.filter((transactions_train_df["t_dat"] >= "2018-09-20") & (transactions_train_df["t_dat"] <= "2018-12-31"))\
                                              .groupBy(["customer_id"])\
                                              .count()
df_2018 = transaction_train_2018.join(customers_df, transaction_train_2018.customer_id == customers_df.customer_id, "left")\
                                .toPandas()

In [4]:
transaction_train_2019 = transactions_train_df.filter((transactions_train_df["t_dat"] >= "2019-09-20") & (transactions_train_df["t_dat"] <= "2019-12-31"))\
                                              .groupBy(["customer_id"])\
                                              .count()
df_2019 = transaction_train_2019.join(customers_df, transaction_train_2019.customer_id == customers_df.customer_id, "left")\
                                .toPandas()

In [11]:
df_2018.head()

,customer_id,count,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,8,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,None,None,ACTIVE,NONE,25,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,3,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,None,None,ACTIVE,NONE,24,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
2,0000c97821eb48d0e590fd309133f0a6c08f7750f64ccc...,1,0000c97821eb48d0e590fd309133f0a6c08f7750f64ccc...,None,None,ACTIVE,NONE,49,8ff2d3103bd127d46e7238c6cee0f3fd9a01bd4a49fb98...
3,0001f8cef6b9702d54abf66fd89eb21014bf98567065a9...,7,0001f8cef6b9702d54abf66fd89eb21014bf98567065a9...,None,None,ACTIVE,NONE,21,4211d6bf468de190b2f135feed96ed94b1c47f1d68991c...
4,000226b9ea81019249060b376b516f821a80e9b24f89a7...,12,000226b9ea81019249060b376b516f821a80e9b24f89a7...,None,None,ACTIVE,NONE,25,7bc2c8dd1064b51b2c0d41bdf6a5d8224bd1b16fbeaf6d...


In [152]:
df_2018_columns = ["customer_id", "count", "customer_id_delete", "FN", "Active", "club_member_status", "fashion_news_frequency", "age", "postal_code"]
df_2018.columns = df_2018_columns
df_2018 = df_2018.drop(columns=["customer_id_delete"])
df_2018 = df_2018[["customer_id", "FN", "Active", "club_member_status", "fashion_news_frequency", "age", "postal_code", "count"]]

In [101]:
df_2018

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,count
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,None,None,ACTIVE,NONE,25,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...,8
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,None,None,ACTIVE,NONE,24,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...,3
2,0000c97821eb48d0e590fd309133f0a6c08f7750f64ccc...,None,None,ACTIVE,NONE,49,8ff2d3103bd127d46e7238c6cee0f3fd9a01bd4a49fb98...,1
3,0001f8cef6b9702d54abf66fd89eb21014bf98567065a9...,None,None,ACTIVE,NONE,21,4211d6bf468de190b2f135feed96ed94b1c47f1d68991c...,7
4,000226b9ea81019249060b376b516f821a80e9b24f89a7...,None,None,ACTIVE,NONE,25,7bc2c8dd1064b51b2c0d41bdf6a5d8224bd1b16fbeaf6d...,12
...,...,...,...,...,...,...,...,...
581181,ffdaf8a5d0d4bbe3511d349ba82382a97232377c0ab9da...,1.0,1.0,ACTIVE,Regularly,66,1be5b3fdb353ffc4a33ac8aa89242f587d6cd7fef42337...,2
581182,ffe3df5c55a419ed1d1a3ee06539658e9f3e6b8ebc0069...,None,None,ACTIVE,NONE,33,6e543be9c1e7654e2326ba7fa64ffd2e42da8da23cc4af...,4
581183,ffebe72466db90d0effde7ff04bf5edd17c7ab2ea9a8fd...,1.0,1.0,ACTIVE,Regularly,38,715494c611efff2348c27dcac148354099d3c5159a5ee6...,6
581184,fff63d0371da299d55e8f71115eb6f9c484e05c9cc43ff...,1.0,1.0,ACTIVE,Regularly,28,7c22218bbcfdac3b3c507708f749a2fa9dea383a103093...,3


In [153]:
import numpy as np

df_2018["FN"] = df_2018["FN"].replace('1.0', True)
df_2018["FN"] = df_2018["FN"].replace(np.nan, False)
print(df_2018["FN"].value_counts())

df_2018["Active"] = df_2018["Active"].replace('1.0', True)
df_2018["Active"] = df_2018["Active"].replace(np.nan, False)
print(df_2018["Active"].value_counts())

cleanup = {"club_member_status": {"ACTIVE": 1, "PRE-CREATE": 2, "LEFT CLUB": 3}, 
          "fashion_news_frequency": {"NONE": 1, "Regularly": 2, "Monthly": 3}}
df_2018 = df_2018.replace(cleanup)
print(df_2018["club_member_status"].value_counts())
print(df_2018["fashion_news_frequency"].value_counts())
df_2018["club_member_status"] = df_2018["club_member_status"].fillna(0)
df_2018["fashion_news_frequency"] = df_2018["fashion_news_frequency"].fillna(1)
df_2018["club_member_status"] = df_2018["club_member_status"].astype(str).astype(float)#.astype("category")
df_2018["fashion_news_frequency"] = df_2018["fashion_news_frequency"].astype(str).astype(float)#.astype("category")


df_2018["age"] = df_2018["age"].astype(float)
mean_age = df_2018["age"].mean()
df_2018["age"] = df_2018["age"].fillna(mean_age)

df_2018["postal_code"] = df_2018["postal_code"].astype("category")

False    353868
True     227318
Name: FN, dtype: int64
False    360079
True     221107
Name: Active, dtype: int64
1.0    545250
2.0     32396
3.0       187
Name: club_member_status, dtype: int64
1.0    349898
2.0    227554
3.0       492
Name: fashion_news_frequency, dtype: int64


In [103]:
df_2018.dtypes

customer_id                 object
FN                            bool
Active                        bool
club_member_status         float64
fashion_news_frequency     float64
age                        float64
postal_code               category
count                        int64
dtype: object

In [165]:
df_2018_train = df_2018[["FN", "Active", "club_member_status", "fashion_news_frequency", "age", "count"]]
X = df_2018_train[["FN", "Active", "club_member_status", "fashion_news_frequency", "age"]]
y = df_2018_train["count"]

In [148]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [117]:
from sklearn.linear_model import LinearRegression


clf_linear_regression = LinearRegression().fit(X_train, y_train)

In [118]:
predictions_linear_regression = clf_linear_regression.predict(X_test)
predictions_linear_regression

array([8.24803601, 7.50064945, 7.55102592, ..., 8.24173895, 8.05282718,
       7.54472886])

In [119]:
from sklearn.metrics import mean_squared_error, r2_score

print("Mean squared error: %.2f" % mean_squared_error(y_test, predictions_linear_regression))
print("Coefficient of determination: %.2f" % r2_score(y_test, predictions_linear_regression))

Mean squared error: 91.58
Coefficient of determination: 0.01


In [142]:
import tensorflow as tf 
print("TensorFlow version:", tf.__version__)

SHUFFLE_BUFFER = 500
BATCH_SIZE = 2

TensorFlow version: 2.8.0


In [166]:
df_2018_train.dtypes

FN                           bool
Active                       bool
club_member_status        float64
fashion_news_frequency    float64
age                       float64
count                       int64
dtype: object

In [167]:
target = df_2018_train.pop("count")

In [168]:
df_2018_train=np.asarray(df_2018_train).astype('float32')
df_2018_train=tf.convert_to_tensor(df_2018_train)

In [169]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(df_2018_train)

In [161]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

normalizer(df_2018_train.iloc[:3])

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'iloc'

In [170]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [171]:
model = get_basic_model()
model.fit(df_2018_train, target, epochs=15, batch_size=BATCH_SIZE)

Epoch 1/15
290593/290593 [==============================] - 389s 1ms/step - loss: -1230286208.0000 - accuracy: 0.1515
Epoch 2/15
290593/290593 [==============================] - 399s 1ms/step - loss: -17423955968.0000 - accuracy: 0.1516
Epoch 3/15
290593/290593 [==============================] - 381s 1ms/step - loss: -74206814208.0000 - accuracy: 0.1516
Epoch 4/15
290593/290593 [==============================] - 376s 1ms/step - loss: -198158155776.0000 - accuracy: 0.1516
Epoch 5/15
290593/290593 [==============================] - 405s 1ms/step - loss: -415292850176.0000 - accuracy: 0.1516
Epoch 6/15
290593/290593 [==============================] - 384s 1ms/step - loss: -750854471680.0000 - accuracy: 0.1516
Epoch 7/15
290593/290593 [==============================] - 383s 1ms/step - loss: -1233448206336.0000 - accuracy: 0.1516
Epoch 8/15
290593/290593 [==============================] - 390s 1ms/step - loss: -1887607062528.0000 - accuracy: 0.1516
Epoch 9/15
290593/290593 [================

KeyboardInterrupt: 